# Python Notebooks on Microsoft Fabric

In [ ]:
%%configure
{
    "defaultLakehouse": {
        "name": "lh_fmdk_solution_config",
    },
}

In [ ]:
import logging
import notebookutils as nbutils
import pandas as pd
import pyarrow as pa
import sempy
import sempy.fabric as fabric
import sys
import time
from typing import Any

In [ ]:
def get_stdout_logger(logger_name: str, silence_other_loggers=True) -> logging.Logger:
    """
    Return a named stdout Logger object.

    Args:
        logger_name (str): Name of logger
        silence_other_loggers (bool, optional): Silences other loggers. Defaults to True.

    Returns:
        logging.Logger: A named Logger object
    """
    # Set general logging level for all loggers
    logging.basicConfig(level=logging.WARNING)

    if silence_other_loggers:
        # Silence all other loggers
        for name, logger in logging.root.manager.loggerDict.items():
            if name != f"{logger_name}":
                logging.getLogger(name).setLevel(logging.WARNING)

    # Use a named logger
    nb_logger = logging.getLogger(logger_name)
    nb_logger.setLevel(logging.INFO)

    # Add handler and formattter
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter("%(asctime)s %(levelname)-5s %(message)s", datefmt="%H:%M:%S")

    handler.setFormatter(formatter)
    nb_logger.addHandler(handler)
    nb_logger.propagate = False

    return nb_logger

In [ ]:
def excel_sheet_to_pyarrow_table(abfss_path: str, sheet_name: str, logger: logging.Logger=None,  **kwargs: Any) -> pa.Table:
    """
    Read a specific sheet from an Excel file.
    
    Args:
        abfss_path: ABFSS path to the Excel file (.xlsx format)
        sheet_name: Name of the sheet to read
        
    Returns:
        PyArrow Table containing the sheet data
        
    Example:
        >>> config_table = read_config_sheet(
        ...     "abfss://container@storage.dfs.core.windows.net/config/solution_config.xlsx",
        ...     "workspaces"
        ... )
    """
    try:
        # Read Excel sheet using pandas
        df = pd.read_excel(
            abfss_path,
            sheet_name=sheet_name,
            engine='openpyxl'  # Required for .xlsx files
        )
        
        # Convert to PyArrow Table for efficient columnar storage
        arrow_table = pa.Table.from_pandas(df)

        if logger:
            logger.info(f"✓ Successfully read sheet '{sheet_name}' with {len(df)} rows")
            logger.info(f"  Columns: {', '.join(df.columns)}")
        
        return arrow_table
        
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {abfss_path}")
    except ValueError as e:
        raise ValueError(f"Sheet '{sheet_name}' not found in workbook: {e}")
    except Exception as e:
        raise Exception(f"Error reading Excel file: {str(e)}")

In [ ]:
def create_workspaces(workspaces_table: pa.Table, capacity_id: str, logger: logging.Logger = None) -> pa.Table:
    """
    Create workspaces from a PyArrow table and add workspace IDs as a new column.
    
    Args:
        workspaces_table: PyArrow table with workspace configurations
        capacity_id: Azure capacity ID for workspace creation
        logger: Logger object for logging operations
        
    Returns:
        PyArrow table with added 'workspace_id' column
    """
    try:
        # Convert to list for iteration
        workspace_list = workspaces_table.to_pylist()
        
        # List to hold the returned id values for workspaces
        workspace_ids = []

        # Iterate and create workspaces
        for ix, workspace in enumerate(workspace_list):
            try:
                ws_id = fabric.create_workspace(
                    display_name=workspace["workspace_name"],
                    capacity_id=capacity_id,
                    description=workspace["workspace_description"]
                )
                
                time.sleep(3)
                workspace_ids.append(ws_id)
                
                if logger:
                    logger.info(f"✓ Created Workspace - Name: {workspace['workspace_name']}, Id={ws_id}")
                    
            except Exception as e:
                # Log error and append None for failed workspace creation
                workspace_ids.append(None)
                if logger:
                    logger.error(f"✗ Failed to create workspace '{workspace['workspace_name']}': {str(e)}")
                else:
                    print(f"Error creating workspace '{workspace['workspace_name']}': {str(e)}")
        
        # Add workspace_id column to the table
        workspace_id_array = pa.array(workspace_ids)
        updated_table = workspaces_table.add_column(workspaces_table.num_columns, 'workspace_id', workspace_id_array)
        
        if logger:
            successful_creates = sum(1 for ws_id in workspace_ids if ws_id is not None)
            logger.info(f"✓ Workspace creation completed: {successful_creates}/{len(workspace_list)} successful")
        
        return updated_table
        
    except Exception as e:
        error_msg = f"Critical error in create_workspaces function: {str(e)}"
        if logger:
            logger.error(error_msg)
        else:
            print(error_msg)
        raise


In [ ]:
CONFIG_ABFSS_PATH = "abfss://ws_fmdk_solution@onelake.dfs.fabric.microsoft.com/lh_fmdk_solution_config.Lakehouse/Files/config/solution_config.xlsx"

CAPID = "AF921006-7E8B-486D-86C9-270A46293624"

# Get a stdout logger object
nb_logger = get_stdout_logger("notebook_logger")

sheet_table = excel_sheet_to_pyarrow_table(CONFIG_ABFSS_PATH, "workspaces", nb_logger)

workspace_list = sheet_table.to_pylist()

for workspace in workspace_list:
    ws_id = fabric.create_workspace(
        display_name=workspace["workspace_name"],
        capacity_id=CAPID,
        description=workspace["workspace_description"]
        )

    time.sleep(3)   

    nb_logger.info(f"Created Workspace - Name: {workspace['workspace_name']}, Id={ws_id}")
    


    # Use the function
    updated_workspace_table = create_workspaces(sheet_table, CAPID, nb_logger)